In [ ]:
# Load environment variables from .env file
load_dotenv()
 
# Retrieve credentials from environment variables
SPACE_ID = os.getenv("ARIZE_SPACE_ID")
API_KEY = os.getenv("ARIZE_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
 
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
 
# Import open-telemetry dependencies
from arize.otel import register
 
# Import open-telemetry dependencies
from arize.otel import register
 
# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,  # in app space settings page
    api_key=API_KEY,  # in app space settings page
    project_name="tracing-haiku-tutorial",  # name this to whatever you would like
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.openai import OpenAIInstrumentor
 
# Finish automatic instrumentation
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
 
ARIZE_SPACE_ID=U3BhY2U6OTEzMDorSUlR
ARIZE_API_KEY=7b6cbce7345fa137c18

In [ ]:
# Load environment variables from .env file
load_dotenv()

SPACE_ID = os.getenv("ARIZE_SPACE_ID")
API_KEY = os.getenv("ARIZE_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

In [ ]:

# Import open-telemetry dependencies
from arize.otel import register

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="tracing-qa-tutorial-azure-rag",
)

# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.langchain import LangChainInstrumentor

# Finish automatic instrumentation
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

# Initialize AzureChatOpenAI client with token limit
azure_client = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # Adjust to your deployment
    api_version="2024-05-01-preview",
    max_tokens=1500, # Limit response length
)



In [ ]:
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate


# Retrieve credentials from environment variables

# Define RAG retrieval tool
@tool
def retrieve_context(query: str) -> str:
    """Retrieve relevant context for a given query from the document knowledge base."""
    try:
        results = vector_store.similarity_search_with_score(query, k=2)
        context = ""
        for doc, score in results:
            if score < 0.3:  # Threshold for relevance
                continue
            context += doc.page_content + "\n"
        logger.info(f"Retrieved context for query '{query}': {context}")
        return context if context else "No relevant context found."
    except Exception as e:
        logger.error(f"Error retrieving context: {e}")
        return "Error retrieving context."

# Define answer generation tool
@tool
def generate_answer(query: str, context: str) -> str:
    """Generate an answer to the query using the provided context."""
    try:
        prompt = f"You are an intelligent assistant. Answer the query concisely using the provided context.\n\nContext: {context}\n\nQuery: {query}"
        response = azure_client.invoke([HumanMessage(content=prompt)])
        logger.info(f"Generated answer for query '{query}': {response.content}")
        return response.content
    except Exception as e:
        logger.error(f"Error generating answer: {e}")
        return "Error generating answer."

# Define the agent prompt template
AGENT_PROMPT = PromptTemplate.from_template("""
You are an intelligent assistant that answers user questions using a document knowledge base. Follow these steps exactly:

1. Understand the user's query.
2. Use the 'retrieve_context' tool to get relevant information.
3. Use the 'generate_answer' tool with the query and context to provide a concise answer.
4. Return the final answer.

Tools: {tools}
Tool names: {tool_names}

Format your response as:
Thought: [Your reasoning]
Action: [Tool name]
Action Input: [Input to the tool]

Current conversation:
{agent_scratchpad}

User input: {input}
""")

# Create the agent
tools = [retrieve_context, generate_answer]
agent = create_react_agent(
    llm=azure_client,
    tools=tools,
    prompt=AGENT_PROMPT
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,  # Limit iterations
)




In [ ]:
from openinference.instrumentation.langchain import LangChainInstrumentor
# Existing tracer provider setup
tracer_provider = register(
    space_id=SPACE_ID,
    api_key=API_KEY,
    project_name="claim_insurance_test_v1",
    batch=True
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)